## Just... hangin around!

In [8]:
import xarray as xr
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


## Opening Zipcode Data

Provide the path to a CSV file of AZ zipcodes and their (lat, lon) coords, or if that file doesn't exist yet, provide the desired location for it.

If the file doesn't exist, it will be created by filtering the `all_zipcodes_path` for zipcodes in AZ and then writing the data to `az_zipcodes_path`.

In [9]:
az_zipcodes_path = pathlib.Path('../data/zipcodes/az_zipcodes.csv')
all_zipcodes_path = pathlib.Path('../data/zipcodes/zip_code_database.xlsx')

if az_zipcodes_path.exists():
    print(f"AZ zipcodes already exists, reading from {az_zipcodes_path.resolve()}")
    az_zipcodes = pd.read_csv(az_zipcodes_path)
    
else:
    print(f"AZ zipcodes does not exist, creating from {all_zipcodes_path.resolve()}")
    df = pd.read_excel(all_zipcodes_path, dtype={'zip': str})
    df = df[df['state'] == "AZ"]
    df = df[['zip', 'county', 'state', 'latitude', 'longitude']]
    df.to_csv(az_zipcodes_path, index=False)
    print(f"AZ zipcodes created at {az_zipcodes_path.resolve()}")
    
print(f"AZ zipcodes sample:")
az_zipcodes

AZ zipcodes already exists, reading from /Users/joshuaelms/Desktop/github_repos/az-temp-data/data/zipcodes/az_zipcodes.csv
AZ zipcodes sample:


,zip,county,state,latitude,longitude
0,85001,Maricopa County,AZ,33.45,-112.06
1,85002,Maricopa County,AZ,33.45,-112.06
2,85003,Maricopa County,AZ,33.45,-112.08
3,85004,Maricopa County,AZ,33.44,-112.07
4,85005,Maricopa County,AZ,33.44,-112.12
...,...,...,...,...,...
564,86544,Apache County,AZ,36.61,-109.16
565,86545,Apache County,AZ,36.66,-109.60
566,86547,Apache County,AZ,36.53,-109.44
567,86555,Cochise County,AZ,31.38,-109.55


## Opening Weather Data .nc File

First open the file, then subset by lat/lon to extract a bounding box around Arizona (or whichever lat/lons).

In [10]:
sample_path = pathlib.Path('../data/raw/ncdd-202012-grd-scaled.nc')
lat_lims = (30.0, 37.5)
lon_lims = (-115.5, -109)
ds = xr.open_dataset(sample_path)
lats, lons = ds['lat'].values, ds['lon'].values
lat_idxs = np.arange(lats.size)[(lats >= lat_lims[0]) & (lats <= lat_lims[1])]
lon_idxs = np.arange(lons.size)[(lons >= lon_lims[0]) & (lons <= lon_lims[1])]
ds = ds.isel(lat=lat_idxs, lon=lon_idxs)
ds

<xarray.Dataset>
Dimensions:  (time: 31, lat: 180, lon: 156)
Coordinates:
  * time     (time) datetime64[ns] 2020-12-01 2020-12-02 ... 2020-12-31
  * lat      (lat) float32 30.02 30.06 30.1 30.15 ... 37.35 37.4 37.44 37.48
  * lon      (lon) float32 -115.5 -115.4 -115.4 -115.4 ... -109.1 -109.1 -109.0
Data variables:
    tmax     (time, lat, lon) float64 ...
    tmin     (time, lat, lon) float64 ...
    prcp     (time, lat, lon) float64 ...
    tavg     (time, lat, lon) float64 ...
Attributes: (12/33)
    date_created:               2022-09-02 08:18:05.885934
    date_modified:              2022-09-02 08:18:05.886525
    Conventions:                CF-1.7, ACDD-1.3
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    title:                      nClimGrid-Daily, Gridded Fields
    source:                     GHCN-Daily, nClimGrid-Monthly
    ...                         ...
    time_coverage_start:        2020-12-01
    time_coverage_end:          2020-12-31
    program:                    Center for Weather and Climate/Climate Scienc...
    geospatial_lat_resolution:  0.04167 degrees
    geospatial_lon_resolution:  0.04167 degrees
    platform:                   station

## Match AZ Zipcode Coordinates to nClimGrid-Daily

Each zipcode coordinate will be replaced by its closest coordinate in the .nc weather data. I will not account for the earth's curvature in calculating latitude distance, as it shouldn't be too important in a small region closer to the equator than poles.

In [11]:
zips = az_zipcodes['zip'].values
zip_lat = az_zipcodes['latitude'].values.reshape(1, -1)
zip_lon = az_zipcodes['longitude'].values.reshape(1, -1)
nc_lat = ds['lat'].values.reshape(1, -1)
nc_lon = ds['lon'].values.reshape(1, -1)

nearest_lat_idxs = np.argmin(np.abs(nc_lat.T - zip_lat), axis=0)
nearest_lon_idxs = np.argmin(np.abs(nc_lon.T - zip_lon), axis=0)
nearest_lats = nc_lat[0, nearest_lat_idxs]
nearest_lons = nc_lon[0, nearest_lon_idxs]

## Extract $T_{\text{max}}$ in each zipcode

In [12]:
tmax = ds['tmax']
zip_aligned_tmax = tmax.sel(lat=nearest_lats, lon=nearest_lons, method='nearest')
print(zip_aligned_tmax)

<xarray.DataArray 'tmax' (time: 31, lat: 569, lon: 569)>
[10036591 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2020-12-01 2020-12-02 ... 2020-12-31
  * lat      (lat) float32 33.44 33.44 33.44 33.44 ... 36.65 36.52 31.4 36.27
  * lon      (lon) float32 -112.1 -112.1 -112.1 -112.1 ... -109.4 -109.6 -109.3
Attributes:
    comment:           Values should be rounded to the nearest hundredth. Eac...
    reference:         https://doi.org/10.1175/JTECH-D-22-0024.1
    long_name:         Temperature, daily maximum
    valid_min:         -100.0
    metadata_link:     https://doi.org/10.25921/c4gt-r169
    source:            GHCN-Daily CSV files 
    standard_name:     air_temperature
    units:             degree_Celsius
    valid_max:         100.0
    id:                /workspace/home2/home/imke.durre/active/live/products/...
    naming_authority:  gov.noaa.ncei


In [13]:
zip_aligned_tmax.values

In [ ]:
# # Extract the relevant data
# tmax_data = zip_aligned_tmax.values

# # Create a list of timestamps
# timestamps = zip_aligned_tmax["time"].values

# # Create a grid of latitude and longitude values
# lats = zip_aligned_tmax['lat'].values
# lons = zip_aligned_tmax['lon'].values

# # Create a Cartopy map of Arizona
# # arizona_map = ccrs.Mercator()  # Choose the map projection (Plate Carrée for this example)

# # Iterate through each timestamp and plot the heatmap
# for i in range(len(timestamps)):
#     print("im just bein real slow")
#     tmax = tmax_data[i]

#     # Create the heatmap
#     plt.figure(figsize=(10, 8))
    
#     # Create a Cartopy subplot and add Arizona map as the background
#     ax = plt.axes(projection=arizona_map)
#     ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=arizona_map)
    
#     print("im just bein real slow2")
    
#     # Add state boundaries, rivers, and other features using Cartopy's cfeature
#     ax.add_feature(cfeature.BORDERS, linestyle=':')
#     # You can add more features as needed
    
#     # Highlight NaN values
#     # Create a mask to identify NaN values
#     nan_mask = np.isnan(tmax)
#     tmax[nan_mask] = -30  # Set NaN values to NaN again for the legend
    
#     print("im just bein real slow3")
    
#     # Plot the heatmap on top of the map
#     im = ax.imshow(tmax, cmap='magma', origin='lower', extent=[lons.min(), lons.max(), lats.min(), lats.max()], transform=arizona_map)
    
#     # Add a colorbar
#     cbar = plt.colorbar(im)
#     print("im just bein real slow4")
#     cbar.set_label('Temperature (°C)')
    
#     # Set title and labels
#     plt.title(f'Temperature Heatmap - {timestamps[i]}')
    
#     # Save or display the plot
#     plt.savefig(f'heatmap_{i}.png')  # You can save the plot to a file if needed
#     plt.show()  # Or display it directly
